Copyright 2024 Zhixuan Hu

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [29]:
import numpy as np
import pandas as pd
import os, re, sys, json, time, random
sys.path.append(os.getcwd() + '\\..\\')
from llm_models.gpt_3_5_turbo_0125 import get_chat_response

In [30]:
pattern = re.compile(r".*\.csv")
files = os.listdir('./')
files = [file for file in files if pattern.match(file)]
files

['gpt_0125_00.csv',
 'human_00.csv',
 'human_01_01.csv',
 'human_01_02.csv',
 'questions_00.csv',
 'questions_01_01.csv',
 'questions_01_02.csv',
 'test_gpt_0125.csv',
 'test_human.csv',
 'test_questions.csv']

#### AI回答数据集

In [31]:
ansfile = './test_gpt_0125.csv'
if (os.path.exists(ansfile)):
    ansdf = pd.read_csv(ansfile)
else:
    ansdf = pd.DataFrame(columns=['question_id','answer_id', 'text'])
ansdf

,Unnamed: 0,question_id,answer_id,text
0,0,454,1,深度学习是一种机器学习的方法，它通过模拟人类神经网络的结构和功能来实现对数据的学习和预测。在...
1,1,454,2,深度学习作为一种机器学习方法，其本质是通过神经网络来实现对数据的学习和模式识别。尽管深度学习...
2,2,454,3,深度学习并没有脱离数学，相反，数学在深度学习中扮演着非常重要的角色。深度学习是建立在数学基础...
3,3,455,1,深度学习是一门需要深厚理论基础和丰富实践经验的学科，纯靠造假是无法取得真正的科研成果和发表高...
4,4,455,2,不应该仅仅依靠造假来发表论文。虽然有些人可能会通过欺骗和伪造数据来发表论文，但这种行为是极其...
5,5,455,3,深度学习的发展离不开研究者的创造性和努力，造假并不是一个可持续的方式。深度学习是一个涉及大量...
6,6,456,1,嗯，这个问题确实让人感到困惑。或许是因为深度学习领域的竞争日益激烈，大家都在追求创新和突破，...
7,7,456,2,深度学习顶会公开数据集、代码等资源的初衷是为了促进学术界的交流与合作，以推动科研领域的发展。...
8,8,456,3,可能有以下几个原因：\n\n1. 数据集质量问题：公开数据集可能存在标注错误、样本不足、数据...
9,9,457,1,清华大学和北京大学作为中国最顶尖的高校之一，其学生群体自然也备受瞩目。关于他们是否自信这个问...


#### 鉴定每个问题的已有回答数，并调用api补齐每个问题的回答至num_ans个，每个回答的temperature递增

In [32]:
def seek_max_ansid(df: pd.DataFrame):
    return df.groupby('question_id')['answer_id'].agg('max')

seek_max_ansid(ansdf)

question_id
454    3
455    3
456    3
457    3
458    3
459    3
460    3
461    3
462    3
463    3
Name: answer_id, dtype: int64

In [33]:
qfile = './test_questions.csv'
qdf = pd.read_csv(qfile)
qdf

,Unnamed: 0,question_id,text
0,1140,454,深度学习一定程度上脱离了数学吗？
1,1085,455,深度学习纯靠造假能发论文嘛？
2,707,456,深度学习顶会，为什么公开数据集，代码，也跑不出论文的结果？
3,769,457,清华大学、北京大学的学生，都自信吗？
4,1642,458,清华大学和北京大学各有何优劣？
...,...,...,...
103,687,557,高校老师收入不高，为什么大批海归却会很努力成为高校老师？
104,867,558,魔方的数学理论基础是什么？
105,1950,559,鸿蒙原生应用没有微信QQ，华为着急吗？
106,211,560,鸿蒙系统到底是不是安卓系统?


In [34]:
qids = qdf['question_id'].to_list()
num_ans = 3
max_ansids = seek_max_ansid(ansdf)
p_words = 0.5
p_example = 0.3
p_human = 0.2
for qid in qids:
    try:
        ansid = max_ansids.loc[qid] + 1
    except:
        ansid = 1
    question = qdf.loc[qdf.question_id == qid, 'text'].values[0]
    while ansid <= num_ans:
        temperature = ansid * 0.4
        prompt_words = ""
        prompt_human = ""
        if random.random() < p_words:
            words = random.randint(3, 11) * 100
            if random.random() < p_example:
                prompt_words = f"——请用大约{words}字谈谈你的看法，可适当以你自身经历举例。"
            else:
                prompt_words = f"——请用大约{words}字谈谈你的看法。"
        if random.random() < p_human:
            prompt_human = "！！！请给出个性和人性的回答，使你看起来像一个人类作者，不要说“当然”！！！"
        print(question + prompt_words + prompt_human)
        answer = get_chat_response(question=question + prompt_words + prompt_human, temperature=temperature)[0]
        if answer:
            ansdf.loc[len(ansdf)] = pd.Series({**dict(question_id=qid, answer_id = ansid, text=answer, temperature=temperature), **{'text-embedding-3-small': 'nan'}})
            ansid += 1


现在为什么没有黑客了？——请用大约800字谈谈你的看法。


2024-04-24 10:41:44,225 - INFO - HTTP Request: POST https://api.crowtalk.live/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-24 10:41:44,228 - INFO - AI 响应：ChatCompletion(id='chatcmpl-9HMrcTz63ts7IvKNxZTQ4dK4tBAgl', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='黑客并没有消失，只是他们的形式和活动方式发生了变化。过去，黑客主要是指那些利用计算机技术破坏系统、窃取信息或者进行其他非法活动的人。随着网络安全意识的提高和技术的发展，许多国家和组织都加大了网络安全的监管和打击力度，使得传统意义上的黑客活动受到了限制。\n\n然而，随着互联网的普及和信息技术的发展，黑客活动也在不断演变。现代黑客更多地是指那些具有高超技术能力的计算机专家，他们可能是为了测试系统的安全性而进行攻击，或者是为了揭示网络安全漏洞而进行研究。这些黑客通常被称为“白帽黑客”或“道德黑客”，他们的目的是为了维护网络安全，而不是为了破坏系统或者窃取信息。\n\n另外，随着网络安全技术的不断发展，许多组织和企业也加大了对网络安全的投入，采取了更加严格的安全措施，使得黑客更难以攻击系统。同时，许多国家也加强了对网络犯罪的打击力度，制定了更加严格的法律法规，对黑客进行惩罚和打击。\n\n此外，随着人们对网络安全意识的提高，越来越多的人意识到网络安全的重要性，采取了更加谨慎的上网行为，避免成为黑客攻击的目标。同时，许多互联网公司也加强了对用户信息的保护，加密了用户数据，降低了黑客攻击的可能性。\n\n总的来说，黑客并没有消失，只是他们的形式和活动方式发生了变化。随着网络安全技术的不断发展和监管力度的加大，传统意义上的黑客活动受到了限制，而新型的“白帽黑客”活动逐渐兴起。同时，人们对网络安全意识的提高和谨慎的上网行为也降低了黑客攻击的可能性。因此，虽然黑客活动仍然存在，但是对网络安全的保护和打击力度的加大使得黑客活动变得更加困难和危险。', role='

现在为什么没有黑客了？


2024-04-24 10:41:49,305 - INFO - HTTP Request: POST https://api.crowtalk.live/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-24 10:41:49,308 - INFO - AI 响应：ChatCompletion(id='chatcmpl-9HMrsYcnD3R4pyetxGUkAkU6iue8d', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='黑客仍然存在，只是由于技术的发展和网络安全意识的提高，黑客们变得更加隐秘和难以被发现。此外，许多组织和公司也加强了网络安全措施，以防止黑客入侵。所以虽然黑客仍然存在，但他们的活动可能变得更加隐蔽和难以察觉。', role='assistant', function_call=None, tool_calls=None))], created=1713926508, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=119, prompt_tokens=18, total_tokens=137))


现在为什么没有黑客了？


...

2024-04-24 11:32:58,286 - INFO - HTTP Request: POST https://api.crowtalk.live/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-24 11:32:58,288 - INFO - AI 响应：ChatCompletion(id='chatcmpl-9HNfCbMmatnQ67klSpRnsbIPyC9Db', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='关于黑人需要混几代血才能接近白人肤色这个问题，首先我认为肤色并不是衡量一个人价值和身份的标准。每个人都应该被尊重和平等对待，不论其肤色如何。然而，这个问题也反映了种族歧视和对肤色的偏见，在这个问题上我想从几个方面进行探讨。\n\n首先，肤色是由遗传决定的，混血后代的肤色可能会有所变化，但并不是说混几代就能接近白人肤色。肤色是由多种基因共同决定的，不同人的基因组合会导致不同的肤色，而且肤色也受到环境因素的影响。因此，混血后代的肤色可能会有一定程度的变化，但并不是说混血几代就会变成白人肤色。\n\n其次，肤色并不应该成为人们评判他人的标准。无论是黑人、白人还是其他种族，每个人都应该被尊重和平等对待。种族歧视和对肤色的偏见是不应该存在的，我们应该尊重每个人的独特性和个性，而不是仅仅根据肤色来评判一个人的价值和身份。\n\n最后，我想说一下我个人的经历。我是一个混血儿，我的父亲是白人，母亲是黑人。我的肤色是中等肤色，既不是很黑也不是很白。在我的成长过程中，我曾经遇到过一些种族歧视和对肤色的偏见，但我始终坚信每个人都应该被尊重和平等对待，不论其肤色如何。我也希望通过我的经历，能够帮助更多人认识到肤色并不是评判一个人的标准，每个人都应该被尊重和平等对待。\n\n总的来说，肤色并不应该成为评判一个人的标准，每个人都应该被尊重和平等对待。种族歧视和对肤色的偏见是不应该存在的，我们应该尊重每个人的独特性和个性。希望我们能够建立一个更加包容和平等的社会，让每个人都能够自由地展现自己的个性和才华。', role='assistant', function_call=N

黑人需要混几代血才能接近白人肤色？！！！请给出个性和人性的回答，使你看起来像一个人类作者，不要说“当然”！！！


2024-04-24 11:33:01,995 - INFO - HTTP Request: POST https://api.crowtalk.live/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-24 11:33:01,997 - INFO - AI 响应：ChatCompletion(id='chatcmpl-9HNfQiTclRJObScpf6exhALQAELJp', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='种族和肤色并不代表一个人的性格和人性，每个人都有自己独特的特质和价值，不应该用肤色来衡量一个人的价值。人类应该相互尊重和包容，不论肤色如何，我们都应该团结在一起，共同创造一个和平、包容的社会。所以，关于黑人需要混几代血才能接近白人肤色的问题，并不重要。重要的是我们如何相互理解，接纳和尊重彼此。让我们共同努力，建立一个更加美好的世界。', role='assistant', function_call=None, tool_calls=None))], created=1713929580, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=182, prompt_tokens=64, total_tokens=246))


黑人需要混几代血才能接近白人肤色？


2024-04-24 11:33:05,354 - INFO - HTTP Request: POST https://api.crowtalk.live/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-24 11:33:05,358 - INFO - AI 响应：ChatCompletion(id='chatcmpl-9HNfUHGX7c3KT3RbWxw28R4KPm1ep', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='黑人需要混合多代不同种族的血统才能逐渐接近白人肤色，但这并不意味着黑人需要放弃或改变他们的种族身份。种族多样性是一个美丽的事实，每个人都应该自豪地接受自己的肤色和文化背景。种族间的混合并不意味着一种种族更优越或更美丽，每个种族都有其独特之处。', role='assistant', function_call=None, tool_calls=None))], created=1713929584, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=141, prompt_tokens=29, total_tokens=170))


In [35]:
ansdf

,Unnamed: 0,question_id,answer_id,text
0,0.0,454,1,深度学习是一种机器学习的方法，它通过模拟人类神经网络的结构和功能来实现对数据的学习和预测。在...
1,1.0,454,2,深度学习作为一种机器学习方法，其本质是通过神经网络来实现对数据的学习和模式识别。尽管深度学习...
2,2.0,454,3,深度学习并没有脱离数学，相反，数学在深度学习中扮演着非常重要的角色。深度学习是建立在数学基础...
3,3.0,455,1,深度学习是一门需要深厚理论基础和丰富实践经验的学科，纯靠造假是无法取得真正的科研成果和发表高...
4,4.0,455,2,不应该仅仅依靠造假来发表论文。虽然有些人可能会通过欺骗和伪造数据来发表论文，但这种行为是极其...
...,...,...,...,...
319,NaN,560,2,鸿蒙系统是华为公司自行研发的操作系统，它与安卓系统有所不同，虽然两者都是基于Linux内核开...
320,NaN,560,3,鸿蒙系统和安卓系统是两个不同的操作系统。鸿蒙系统是由华为公司自主研发的操作系统，旨在为物联网...
321,NaN,561,1,关于黑人需要混几代血才能接近白人肤色这个问题，首先我认为肤色并不是衡量一个人价值和身份的标准...
322,NaN,561,2,种族和肤色并不代表一个人的性格和人性，每个人都有自己独特的特质和价值，不应该用肤色来衡量一个...


In [36]:
time_str = time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime())
if os.path.exists(ansfile):
    os.rename(ansfile, './history/' + ansfile + '-' + time_str + '.csv')
ansdf.to_csv(ansfile, encoding='utf-8-sig')